In [1]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import chardet
import glob
import os
import shutil
from fuzzywuzzy import process
import re

# Load separate tables and merge

### E.g., "Broadleaf_deciduous_liana_tropical_utf8.csv"
`if str.startswith('Broadleaf') or str.startswith('Needleleaf')`
- Broadleaf = 'Funtype'
- deciduous = 'Funtype'
- liana = 'Plantform'
- tropical = 'Tregion'

### E.g., "C3_Crop_utf8"
`if str.startswith('C3') or str.startswith('C4')`
- C3 = 'Pathway'
- Crop = 'Plantform'

`if str == 'C4_Grass_andropogon_gerardii_utf8.csv`
- C4 = 'Pathway'
- Grass = 'Plantform'

`if str == 'Cornus_racemosa_Cuyzam_utf8.csv'`
- no additional columns

In [2]:
# get list of files
directory_path = '../data/han_2022'
path_list = glob.glob(f'{directory_path}/*_utf8.csv')
path_list

['../data/han_2022/Broadleaf_deciduous_tree_boreal_utf8.csv',
 '../data/han_2022/Broadleaf_deciduous_liana_tropical_utf8.csv',
 '../data/han_2022/Cornus_racemosa_Cuyzam_utf8.csv',
 '../data/han_2022/Needleaf_evergreen_tree_boreal_utf8.csv',
 '../data/han_2022/C3_Crop_utf8.csv',
 '../data/han_2022/C4_Crop_utf8.csv',
 '../data/han_2022/C4_Grass_andropogon_gerardii_utf8.csv',
 '../data/han_2022/C3_Grass_utf8.csv',
 '../data/han_2022/Broadleaf_deciduous_tree_tropical_utf8.csv',
 '../data/han_2022/C4_Grass_utf8.csv',
 '../data/han_2022/Broadleaf_deciduous_shrub_temperate_utf8.csv',
 '../data/han_2022/Broadleaf_deciduous_tree_temperate_utf8.csv',
 '../data/han_2022/Broadleaf_evergreen_tree_tropical_utf8.csv']

In [3]:
# get file names only
file_list = []
for path in path_list:
    file_list.append(os.path.basename(path))

# treat files differently depending on name
dfs = []
for file, path in zip(file_list, path_list):

    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    words = file.split('_')
    
    if file.startswith('Broadleaf') or file.startswith('Needleleaf'):
        
        # add columns
        df['Funtype'] = words[0] + ' ' + words[1]
        df['Plantform'] = words[2]
        df['Tregion'] = words[3]

        dfs.append(df)

    elif file.startswith('C3') or file.startswith('C4'):

        # add columns
        df['Pathway'] = words[0]
        df['Plantform'] = words[1]

        dfs.append(df)

    elif file == 'Cornus_racemosa_Cuyzam_utf8.csv':

        dfs.append(df)

In [4]:
data = pd.concat(dfs, axis=0)
data

,Species name,Tleaf,LeafReplicateNumber,PARi,CO2R,Ci,Anet,Fm',Fs,VpdL,Press,OxygenPress,Fo,Fm,Funtype,Plantform,Tregion,Pathway
0,Betula alleghaniensis,25.0000,1,1198.8000,399.8693,129.280300,9.095191,414.000,318.00000,1.542979,98.1000,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
1,Betula alleghaniensis,25.0000,1,1000.5000,400.1021,138.184400,8.935258,454.000,331.00000,1.543947,98.1100,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
2,Betula alleghaniensis,25.0000,1,798.4000,400.5123,143.279200,8.685766,524.000,352.00000,1.534511,98.0900,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
3,Betula alleghaniensis,25.0000,1,500.3000,399.5837,157.681400,7.720055,720.000,397.00000,1.492609,98.1100,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
4,Betula alleghaniensis,25.0000,1,298.1000,399.9732,183.647800,6.227400,871.000,401.00000,1.452542,98.1000,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Millettia macrostachya,30.1284,3,299.9130,400.0130,274.907320,10.149162,696.439,284.75048,1.589443,94.4889,21,222.4772,1059.9,Broadleaf evergreen,tree,tropical,NaN
77,Millettia macrostachya,30.1042,3,200.1260,399.9760,297.503196,7.547358,781.289,271.46780,1.583747,94.4825,21,222.4772,1059.9,Broadleaf evergreen,tree,tropical,NaN
78,Millettia macrostachya,30.1021,3,150.0830,400.0290,319.589918,5.904670,806.936,262.81628,1.569007,94.4773,21,222.4772,1059.9,Broadleaf evergreen,tree,tropical,NaN
79,Millettia macrostachya,30.0923,3,100.0920,399.9980,344.800439,3.529413,825.510,254.73880,1.575467,94.4706,21,222.4772,1059.9,Broadleaf evergreen,tree,tropical,NaN


# Add metadata to all_data as 3 multi-headers

In [5]:
# Read the metadata file
metadata = pd.read_csv('../data/han_2022/han_2022_metadata.txt', delimiter=':', names=['Variable', 'Description'])

# Extract units from the Description
metadata['Unit'] = metadata['Description'].str.extract(r'in ([\w\s\-\u00B5%/]+)').fillna(np.nan)

# Clean up the Variable names by stripping whitespace
metadata['Variable'] = metadata['Variable'].str.strip()

# Set specific variable units
metadata.loc[metadata['Variable'] == 'Species name', 'Unit'] = 1
metadata.loc[metadata['Variable'] == 'Tleaf', 'Unit'] = 'degree_C'
metadata.loc[metadata['Variable'] == 'LeafReplicateNumber', 'Unit'] = 1

metadata

,Variable,Description,Unit
0,Species name,species,1
1,Tleaf,leaf temperature in °C,degree_C
2,LeafReplicateNumber,replicate number for individual leaf,1
3,PARi,photosynthetically active radiation in µmol m...,µmol m-2 s-1
4,CO2R,reference CO2 concentration in µmol mol-1,µmol mol-1
5,Ci,intercellular CO2 concentration in µmol mol-1,µmol mol-1
6,Anet,net photosynthesis in µmol m-2 s-1,µmol m-2 s-1
7,Fm',maximum chlorophyll a fluorescence under ligh...,mV
8,Fs,steady-state chlorophyll a fluorescence under...,mV
9,VpdL,leaf-to-air vapour pressure deficit in kPa,kPa


In [6]:
# Check for variables in data that are not in metadata
for col in data.columns.get_level_values(0).to_list():
    if col not in metadata['Variable'].values.tolist():
        # Append a new row to metadata with NaN for Description and Unit
        new_row = pd.DataFrame([[col, np.nan, np.nan]], columns=metadata.columns)  # Use a list of lists
        metadata = pd.concat([metadata, new_row], ignore_index=True)  # Reset index to avoid conflicts

metadata

,Variable,Description,Unit
0,Species name,species,1
1,Tleaf,leaf temperature in °C,degree_C
2,LeafReplicateNumber,replicate number for individual leaf,1
3,PARi,photosynthetically active radiation in µmol m...,µmol m-2 s-1
4,CO2R,reference CO2 concentration in µmol mol-1,µmol mol-1
5,Ci,intercellular CO2 concentration in µmol mol-1,µmol mol-1
6,Anet,net photosynthesis in µmol m-2 s-1,µmol m-2 s-1
7,Fm',maximum chlorophyll a fluorescence under ligh...,mV
8,Fs,steady-state chlorophyll a fluorescence under...,mV
9,VpdL,leaf-to-air vapour pressure deficit in kPa,kPa


In [7]:
metadata.columns = metadata.columns.str.strip()
metadata['Variable'] = metadata['Variable'].str.strip()
new_header = pd.MultiIndex.from_arrays([
    metadata['Variable'],
    metadata['Description'],  # Descriptions
    metadata['Unit']  # Units
])

# Step 2: Apply the multi-level header to all_data
data.columns = new_header
data

Variable,Species name,Tleaf,LeafReplicateNumber,PARi,CO2R,Ci,Anet,Fm',Fs,VpdL,Press,OxygenPress,Fo,Fm,Funtype,Plantform,Tregion,Pathway
Description,species,leaf temperature in °C,replicate number for individual leaf,photosynthetically active radiation in µmol m-2 s-1,reference CO2 concentration in µmol mol-1,intercellular CO2 concentration in µmol mol-1,net photosynthesis in µmol m-2 s-1,maximum chlorophyll a fluorescence under light in mV,steady-state chlorophyll a fluorescence under light in mV,leaf-to-air vapour pressure deficit in kPa,air pressure in kPa,the pressure of oxygen in kPa,minimum chlorophyll a fluorescence under fully dark-adapted conditions in mV,maximum chlorophyll a fluorescence under fully dark-adapted conditions in mV,NaN,NaN,NaN,NaN
Unit,1,degree_C,1,µmol m-2 s-1,µmol mol-1,µmol mol-1,µmol m-2 s-1,mV,mV,kPa,kPa,kPa,mV,mV,NaN,NaN,NaN,NaN
0,Betula alleghaniensis,25.0000,1,1198.8000,399.8693,129.280300,9.095191,414.000,318.00000,1.542979,98.1000,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
1,Betula alleghaniensis,25.0000,1,1000.5000,400.1021,138.184400,8.935258,454.000,331.00000,1.543947,98.1100,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
2,Betula alleghaniensis,25.0000,1,798.4000,400.5123,143.279200,8.685766,524.000,352.00000,1.534511,98.0900,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
3,Betula alleghaniensis,25.0000,1,500.3000,399.5837,157.681400,7.720055,720.000,397.00000,1.492609,98.1100,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
4,Betula alleghaniensis,25.0000,1,298.1000,399.9732,183.647800,6.227400,871.000,401.00000,1.452542,98.1000,21,400.0000,2177.0,Broadleaf deciduous,tree,boreal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Millettia macrostachya,30.1284,3,299.9130,400.0130,274.907320,10.149162,696.439,284.75048,1.589443,94.4889,21,222.4772,1059.9,Broadleaf evergreen,tree,tropical,NaN
77,Millettia macrostachya,30.1042,3,200.1260,399.9760,297.503196,7.547358,781.289,271.46780,1.583747,94.4825,21,222.4772,1059.9,Broadleaf evergreen,tree,tropical,NaN


# Clean up headers, descriptions, and units

In [8]:
def clean_values(values):
    """
    Cleans the given pandas Index (or Series) of string values by:
    - Stripping leading/trailing whitespace
    - Removing stray quotes
    - Normalizing whitespace and commas
    - Joining related terms
    
    Parameters:
        values (pd.Index or pd.Series): The values to be cleaned.

    Returns:
        pd.Index: The cleaned values.
    """
    cleaned_values = (
        values
        .str.strip()  # Remove leading/trailing whitespace
        .str.replace(r"(^['\"]|['\"]$)", '', regex=True)  # Remove leading/trailing quotes
        .str.replace(r"\s*,\s*", ', ', regex=True)  # Normalize commas with single space after them
        .str.replace(r"\s+", ' ', regex=True)  # Normalize whitespace within strings
        .str.replace(r"' '", '', regex=True)  # Remove isolated single quotes
        .str.replace(r"([a-zA-Z]),([a-zA-Z])", r"\1, \2", regex=True)  # Add space after commas if missing
    )

    # Handle cases like ' evergreen, deciduous '
    cleaned_values = cleaned_values.str.replace(r"['‘’]", '', regex=True)  # Remove any stray single quotes
    cleaned_values = cleaned_values.str.replace(r"\b ,\b", ',', regex=True)  # Fix any stray commas

    return cleaned_values

### Units

In [9]:
# clean units
values = data.columns.get_level_values(2)
cleaned_l2_values = clean_values(values)
data.columns = pd.MultiIndex.from_arrays([
    data.columns.get_level_values(0),
    data.columns.get_level_values(1),
    cleaned_l2_values
])

In [10]:
data.columns.get_level_values(2).to_list()

[nan,
 'degree_C',
 nan,
 'µmol m-2 s-1',
 'µmol mol-1',
 'µmol mol-1',
 'µmol m-2 s-1',
 'mV',
 'mV',
 'kPa',
 'kPa',
 'kPa',
 'mV',
 'mV',
 nan,
 nan,
 nan,
 nan]

### Descriptions

In [11]:
# clean descriptions
values = data.columns.get_level_values(1)
cleaned_l1_values = clean_values(values)
data.columns = pd.MultiIndex.from_arrays([
    data.columns.get_level_values(0),
    cleaned_l1_values,
    data.columns.get_level_values(2),
])

In [12]:
data.columns.get_level_values(1).to_list()

['species',
 'leaf temperature in °C',
 'replicate number for individual leaf',
 'photosynthetically active radiation in µmol m-2 s-1',
 'reference CO2 concentration in µmol mol-1',
 'intercellular CO2 concentration in µmol mol-1',
 'net photosynthesis in µmol m-2 s-1',
 'maximum chlorophyll a fluorescence under light in mV',
 'steady-state chlorophyll a fluorescence under light in mV',
 'leaf-to-air vapour pressure deficit in kPa',
 'air pressure in kPa',
 'the pressure of oxygen in kPa',
 'minimum chlorophyll a fluorescence under fully dark-adapted conditions in mV',
 'maximum chlorophyll a fluorescence under fully dark-adapted conditions in mV',
 nan,
 nan,
 nan,
 nan]

### Variables

In [13]:
values = data.columns.get_level_values(0)
cleaned_l0_values = clean_values(values)
data.columns = pd.MultiIndex.from_arrays([
    cleaned_l0_values,
    data.columns.get_level_values(1),
    data.columns.get_level_values(2),
])

In [20]:
# sorted(data.columns.get_level_values(0).to_list())

In [15]:
data.to_csv('data/han_2022/han_2022_data.csv', index=False)